# PyTorch and TensorFlow

一些在学习使用 PyTorch 和 TensorFlow 的时候的笔记。

## 关于维度

我之前一直没理解为什么 PyTorch 里一些 NLP 相关的模块（如 LSTM、GRU）输入一批数据的时候要规定默认数据的维度的第二维是 batch size，而不是把 batch size 放在第一个维度。

例如 GRU 的 input 默认 shape 是 (max_length, batch_size, input_size)，即第一维是每条语句的最大长度（词数量），第二维是批大小，第三位是词嵌入的维度大小。当然，我也可以在传入数据的时候设置参数 `batch_first=True` 这样我就可以传入 (batch_size, max_length, input_size) 的数据了，这样的数据组织方式更符合直观理解。

后来看了 PyTorch Chatbot 的教程才知道，这样做的好处是，如果我想自己在代码里一步一步处理输入序列，每次手动取出一个 time-step，对输入数据取第一个维度的下标的时候，一次就能取到所有 batch 的该下标对应的 time-step。

如果把 batch size 放在第一个维度，那么想取所有 batch 的某一个 time-step 的话，就要用到 Python 的 slice 操作，例如 

In [11]:
import numpy as np
data = np.array([[11, 12, 13],
                 [21, 22, 23],
                 [31, 32, 33]])
print(data[:,1])

[12 22 32]


学习 PyTorch 的 chatbot 教程 中的 attention 的时候，总是对一些输入输出的维度弄不清楚。现在在这里记录一下，假设只看 `forward` 函数，输入的变量是已经 `embedding` 了，在 decoder 中我们是一步一步地处理

```
def forward(self, hidden, encoder_outputs):
    # Calculate the attention weights (energies)
    attn_energies = self.atten(hidden, encoder_outputs)

    # Transpose max_length and batch_size dimensions
    attn_energies_t = attn_energies.t()
    
    # Weights are softmax normalized probability scores (with added dimension)
    attn_energies_t_softmax = F.softmax(attn_energies_t, dim=1)
    atten_weights = attn_energies_t_softmax.unsqueeze(1)

    return atten_weights
```
这里的各个变量的维度应该是这样的：

* hidden.shape: (1, batch_size, hidden_size)
* encoder_outputs.shape: (length, batch_size, embed_size)
* attn_energies.shape: (length, batch_size)
* attn_energies_t.shape: (batch_size, length)
* attn_energies_t_softmax.shape: (batch_size, length)
* atten_weights.shape: (batch_size, 1, length)

在 `return` 的时候把 batch_size 放在第一位的原因是通常会用 `torch.bmm(input, mat2, out=None)` 来把 batch 的 `atten_weights` 和 batch 的 `encoder_outputs` 做 batch matrix-matrix product，这个函数的要求就是 batch_size 是第一个维度。

> If input is a $(b \times n \times m)$ tensor, mat2 is a $(b \times m \times p)$ tensor, out will be a $(b \times n \times p)$ tensor.

因此，在得到了 `atten_weights` 之后，要把 `atten_weights` 应用到 `encoder_outputs` 上时，`encoder_outputs` 也会做一个转置，把 batch_size 放到第一位

```
weighted_sum = torch.bmm(atten_weights, encoder_outputs.transpose(0, 1))
```

此时

* weighted_sum.shape: (batch_size, 1, embed_size)

这时候 `weighted_sum` 的 dimension 就 make sense 了，即 batch_size 个 weighted sum, 每个 weighted sum 的维度是 (1, embed_size)。

当然，如果想用别的方法而不是 `torch.bmm` 来应用 attention 的权重的时候，可以不必把 batch_size 放到第一位。